# Heatmaps

This notebook generates heatmaps for the first or last X fixations based on their durations. You can specify the number of fixations, and the notebook will use that number to analyze the first and last fixations.

The function sorts the fixations by their duration (FixDur) to identify what participants focused on the longest, and then selects the first or last X fixations accordingly.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from math import sqrt, pi,exp, isnan
import os 
root = '../../heatmaps/durations/'

#define the gaussian function for the heatmap (code from Pygaze)
def gaussian(w, h, s):
	W = np.zeros([h,w],dtype=float)
	dims = W.shape
	xo = w/2
	yo = h/2
	for i in range(w):
			for j in range(h):
				W[j,i] = np.exp(-1.0 * (((float(i)-xo)**2/(2*s*s)) + ((float(j)-yo)**2/(2*s*s)) ) )
	return(W)

#loop through the trust data and add gaussians (code from Pygaze)
#make sure to have columns Xloc and Yloc in your dataframe
def create_heatmap(fixations):
	#create empty heatmaps to add the fixation gaussians to
	heatmap = np.zeros(heatmapsize)
	for index, row in fixations.iterrows():
		x = int(strt + row['X'] - int(gwh/2))
		y = int(strt + row['Y'] - int(gwh/2))
		# correct Gaussian size if either coordinate falls outside of
		# display boundaries
		if (not gwh/2 < x < maxX-gwh/2-1) or (not gwh/2 < y < maxY-gwh/2-1):
			hadj=[0,gwh]
			vadj=[0,gwh]
			if 0 > x:
				hadj[0] = abs(x)
				x = 0
			elif maxX < x:
				hadj[1] = gwh - int(x-maxX)
			if 0 > y:
				vadj[0] = abs(y)
				y = 0
			elif maxY < y:
				vadj[1] = gwh - int(y-maxY)
			# add adjusted Gaussian to the current heatmap
			try:
				heatmap[y:y+vadj[1],x:x+hadj[1]] += gaus[vadj[0]:vadj[1],hadj[0]:hadj[1]] 
			except:
			# fixation was probably outside of display
				pass
		else:				
		# add Gaussian to the current heatmap
			heatmap[y:y+gwh,x:x+gwh] += gaus 

	# resize heatmap (Pygaze)
	heatmap = heatmap[strt:maxY+strt,strt:maxX+strt]

	# remove zeros
	lowbound = np.mean(heatmap[heatmap>0])
	heatmap[heatmap<lowbound] = np.nan
	return heatmap




In [2]:
def heatmap_exp(exp, image, scene, amount=10):
    heatmap_head(exp, image, scene, amount)
    heatmap_tail(exp, image, scene, amount)

In [3]:
def heatmap_head(exp, image, scene, amount=10):
    
    subfolder = f'{amount}_longest_duration/scene{scene}_{image}/'
    os.makedirs(root+subfolder, exist_ok=True)
    fix_control = fix[(fix['experience'] == exp)]
    fix_firstx = fix_control.sort_values(by='FixDur', ascending=False).groupby('Participant').head(amount)
    heatmap = create_heatmap(fix_firstx)
    
    
    #plot the heatmap and save the image
    fig = plt.figure()
    imgplot = plt.imshow(background[startX:startX+dims[0], startY:startY+dims[1]])
    imgplot = plt.imshow(heatmap[startX:startX+dims[0], startY:startY+dims[1]], cmap='hot', alpha=0.5)
    plt.axis([0,dims[1],dims[0],0])
    plt.xticks([])
    plt.yticks([])
    
    plt.draw()
    plt.savefig(f'{root}{subfolder}heatmap_{exp}.png', bbox_inches='tight', pad_inches=0)
    
    plt.close(fig)

In [4]:
def heatmap_tail(exp, image, scene, amount=10):
    
    subfolder = f'{amount}_least_durations/scene{scene}_{image}/'
    os.makedirs(root+subfolder, exist_ok=True)
    fix_control = fix[(fix['experience'] == exp)]
    fix_lastx = fix_control.sort_values(by='FixDur', ascending=False).groupby('Participant').tail(amount)
    heatmap = create_heatmap(fix_lastx)
    
    
    #plot the heatmap and save the image
    fig = plt.figure()
    imgplot = plt.imshow(background[startX:startX+dims[0], startY:startY+dims[1]])
    imgplot = plt.imshow(heatmap[startX:startX+dims[0], startY:startY+dims[1]], cmap='hot', alpha=0.5)
    plt.axis([0,dims[1],dims[0],0])
    plt.xticks([])
    plt.yticks([])
    
    plt.draw()
    plt.savefig(f'{root}{subfolder}heatmap_{exp}.png', bbox_inches='tight', pad_inches=0)
    
    plt.close(fig)

In [ ]:
from PIL import Image

pad = "../../data/csv/"
image_pad =  "../../data/Images/CrimeScenes/"

scenes = [1, 2, 3]
images = [1, 2]
experiences = ['CSI', 'Control', 'FirstYear', 'ThirdYear']
amounts = [1, 10, 25, 50]

for image in images:
    for scene in scenes:
        
        img = Image.open(image_pad + f'scene_{scene}_{image}.png')
        width, height = img.size
        
        #size of the display
        maxX = width
        maxY = height
        	
        
        #width of the kernel
        gwh = int(maxX/25)
        
        filename = image_pad + f'scene_{scene}_{image}.png' #background image
        
        fh1 = open(pad + 'fix_evidence.csv', 'r') #open the fixations for trust task
        fix_full = pd.read_csv(fh1, sep=',')
        
        fix = fix_full[(fix_full['Image'] == image) & (fix_full['Scene'] == scene)]
        
        #set the background image for the image to show (jpg)
        background = np.zeros((maxY,maxX,3),dtype=np.float32)
        #layer=255*np.ones((maxY,maxX),dtype=np.float32)
        #background[:,:,3]=layer
        
        #load the background image
        im = np.array(mpimg.imread(filename))
        dims = im.shape
        print(dims)
        
        #find where to place image in background (center of image)
        startX=int(round((maxY-dims[0])/2))
        startY=int(round((maxX-dims[1])/2))
        
        #put image on background
        background[startX:startX+dims[0], startY:startY+dims[1],:] = im
        
        #create the kernel (Pygaze)
        gaus = gaussian(int(gwh), int(gwh), int(gwh/6))
        strt = int(gwh/2)
        heatmapsize = maxY + 2*strt, maxX + 2*strt

        for exp in experiences:
            for amount in amounts:
                heatmap_exp(exp, image, scene, amount)

(768, 713, 3)
(768, 1024, 3)
(768, 1024, 3)
(768, 1024, 3)
(768, 1024, 3)
(768, 1024, 3)
